In [1]:
import os
from langchain.llms import LlamaCpp
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import LlamaCppEmbeddings, OpenAIEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

loader = WebBaseLoader("https://news.ltn.com.tw/news/world/breakingnews/4382800")
data = loader.load()

In [2]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 10,length_function = len)
all_splits = text_splitter.split_documents(data)

In [3]:
# LlamaCppEmbeddings 目前有BUG造成運行速度緩慢，先用OpenAIEmbeddings代替 https://github.com/langchain-ai/langchain/pull/5066
model_path = "/home/sung/llm/chinese-alpaca-2-7b/gml-model-q4_0.bin"
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=LlamaCppEmbeddings(model_path=model_path))

In [5]:
os.environ['OPENAI_API_KEY'] = ''
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [6]:
llama = LlamaCpp(
            model_path=model_path,
            n_gpu_layers=40,
            n_batch=8,
            verbose=True,
            n_ctx = 2048
        )

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 2060 SUPER, compute capability 7.5
llama.cpp: loading model from /home/sung/llm/chinese-alpaca-2-7b/gml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 55296
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 5504
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_int

In [7]:
template =  """
使用以下上下文來回答最後的問題。
如果你不知道答案，就說你不知道，不要試圖編造答案。
最多使用三個句子，並儘可能保持答案簡潔。
{context}
問題：{question}
有用的答案：
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llama,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True
)

In [8]:
question = "LK-99 為什麼只能提供理論上的支持"
result = qa_chain({"query": question})
result["result"]


llama_print_timings:        load time =   250.67 ms
llama_print_timings:      sample time =   251.43 ms /   198 runs   (    1.27 ms per token,   787.49 tokens per second)
llama_print_timings: prompt eval time = 47336.39 ms /  1467 tokens (   32.27 ms per token,    30.99 tokens per second)
llama_print_timings:        eval time =  5703.19 ms /   197 runs   (   28.95 ms per token,    34.54 tokens per second)
llama_print_timings:       total time = 55063.60 ms


'1. 材料合成的難度很高，因此僅能提供理論上的支援。\n2. LK-99是一種超導體材質，需要在高溫和高壓下才能顯現其超導效應。目前尚未有可靠的方法可以同時控制室溫常壓下的LK-99材料的合成和處理，因此仍需進一步研究以確保實際上的運用可行性。\n3. LK-99是一種新型材料，雖然已經被證明在理論上有超導效應存在，但目前還沒有人能夠制備出符合預期條件的LK-99材質。因此，需要進一步的研究和努力才能確定其實際運用可行性。\n4. 現階段對於LK-99材料的合成方法仍然了解不多，所以仍需進行更多的研究和探究以找出有效的合成方法。'

In [9]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True
)
result = qa_chain({"query": question})
result["result"]

'LK-99只能提供理論上的支持，因為它的合成和排列需要非常特殊的條件和位置，並且只有很小一部分晶體能夠實現這種排列。這使得LK-99的合成和重現變得非常困難。'